In [1]:
#get dependencies 
import requests
import json
import pandas as pd
import numpy as np
import pprint
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
#start a new DataFrame
columns=["Laureate ID", "Full Name","Gender","Birthdate/Date Founded", "Country","Continent","Age/Years of Operation", "Year Awarded","Date Awarded", "Award Category", "Awarded For"]
Nobel_Laureates=pd.DataFrame(columns=columns)
Nobel_Laureates

,Laureate ID,Full Name,Gender,Birthdate/Date Founded,Country,Continent,Age/Years of Operation,Year Awarded,Date Awarded,Award Category,Awarded For


In [3]:
#define the url for the api we are connecting to. '?limit=5000' was added at the end to circumvent 25 response limit. 
url="https://api.nobelprize.org/2.0/laureates?limit=5000"
#send a request and format the response as json
JSONContent=requests.get(url).json()
#dump the data so we can see it in a more readable way
content=json.dumps(JSONContent, indent = 4, sort_keys=True)

In [4]:
for x in range(len(JSONContent["laureates"])):
    laureate=JSONContent['laureates'][x]
    Laureate_id=int(laureate["id"])
    try:
        Laureate_name=laureate["fullName"]['en']
        gender=laureate["gender"].capitalize()
        birth_date=laureate["birth"]["date"]
        birth_place=""
        birth_continent=""
        try:
            birth_place=laureate["birth"]["place"]["countryNow"]["en"]
            birth_continent=laureate["birth"]["place"]["continent"]["en"]
        except:
            pass
    except:
        Laureate_name=laureate["orgName"]["en"]
        gender="Organization"
        birth_date=""
        try:
            birth_date=laureate["founded"]["date"]
        except:
            pass
        birth_place=""
        try:
            birth_place=laureate["founded"]["place"]["countryNow"]["en"]
        except:
            pass
        birth_continent=""
        try:
            birth_continent=laureate["founded"]["place"]["continent"]["en"]
        except:
            pass
    
    
    for n in range(len(laureate["nobelPrizes"])):
        award=laureate["nobelPrizes"][n]
        award_year=award["awardYear"]
        award_date=""
        try:
            award_date=award["dateAwarded"]
        except:
            pass
        award_category=award["category"]["en"]
        award_for=award["motivation"]['en']
        age=""
        try:
            age=relativedelta(datetime.strptime(award_date , '%Y-%m-%d'), datetime.strptime(birth_date , '%Y-%m-%d')).years
        except:
            try:
                age=int(award_year)-int(birth_date.split("-")[0])
            except:
                pass
        #append DataFrame with award
        Nobel_Laureates=Nobel_Laureates.append({
            "Laureate ID": Laureate_id,
            "Full Name": Laureate_name,
            "Gender": gender,
            "Birthdate/Date Founded": birth_date, 
            "Country": birth_place, 
            "Continent": birth_continent,
            "Age/Years of Operation": age,
            "Year Awarded": award_year,
            "Date Awarded": award_date, 
            "Award Category": award_category,
            "Awarded For": award_for
        }, ignore_index=True)

In [5]:
Nobel_Laureates=Nobel_Laureates.sort_values("Laureate ID")
Nobel_Laureates.reset_index(drop=True, inplace=True)
Nobel_Laureates

,Laureate ID,Full Name,Gender,Birthdate/Date Founded,Country,Continent,Age/Years of Operation,Year Awarded,Date Awarded,Award Category,Awarded For
0,1,Wilhelm Conrad Röntgen,Male,1845-03-27,Germany,Europe,56,1901,,Physics,in recognition of the extraordinary services h...
1,2,Hendrik Antoon Lorentz,Male,1853-07-18,the Netherlands,Europe,49,1902,,Physics,in recognition of the extraordinary service th...
2,3,Pieter Zeeman,Male,1865-05-25,the Netherlands,Europe,37,1902,,Physics,in recognition of the extraordinary service th...
3,4,Antoine Henri Becquerel,Male,1852-12-15,France,Europe,51,1903,,Physics,in recognition of the extraordinary services h...
4,5,Pierre Curie,Male,1859-05-15,France,Europe,44,1903,,Physics,in recognition of the extraordinary services t...
5,6,"Marie Curie, née Sklodowska",Female,1867-11-07,Poland,Europe,36,1903,,Physics,in recognition of the extraordinary services t...
6,6,"Marie Curie, née Sklodowska",Female,1867-11-07,Poland,Europe,44,1911,,Chemistry,in recognition of her services to the advancem...
7,8,Lord Rayleigh (John William Strutt),Male,1842-11-12,United Kingdom,Europe,62,1904,,Physics,for his investigations of the densities of the...
8,9,Philipp Eduard Anton von Lenard,Male,1862-06-07,Slovakia,Europe,43,1905,,Physics,for his work on cathode rays
9,10,Joseph John Thomson,Male,1856-12-18,United Kingdom,Europe,50,1906,,Physics,in recognition of the great merits of his theo...


In [6]:
Nobel_Laureates.to_csv("Output/Nobel_Laureates.csv", index=False)